In [1]:
import math

In [2]:
# ==========================
# 1. FAKE DATA
# ==========================

tutors = [
    {
        "id": 1,
        "name": "Nguyễn Văn A",
        "subjects": ["Toán"],
        "grades": [8, 9],
        "lat": 21.01,
        "lng": 105.80,
        "hourly_rate": 150_000,
        "profile_completed": 0.9,
        "is_verified_id": True,
        "is_verified_student_card": True,
        "total_classes": 5,
        "completed_classes": 5,
        "avg_rating": 4.8,
        "avg_response_time_sec": 300,   # 5 phút
        "policy_violations_count": 0,
        "free_slots": ["Mon_19-21", "Wed_19-21"],
        # mô phỏng semantic score: càng cao càng hợp mô tả yêu cầu
        "semantic_strength": 0.95
    },
    {
        "id": 2,
        "name": "Trần Thị B",
        "subjects": ["Toán"],
        "grades": [6, 7, 8, 9],
        "lat": 21.03,
        "lng": 105.81,
        "hourly_rate": 130_000,
        "profile_completed": 0.8,
        "is_verified_id": True,
        "is_verified_student_card": False,
        "total_classes": 3,
        "completed_classes": 2,
        "avg_rating": 4.2,
        "avg_response_time_sec": 1800,  # 30 phút
        "policy_violations_count": 0,
        "free_slots": ["Mon_18-20", "Wed_19-21"],
        "semantic_strength": 0.80
    },
    {
        "id": 3,
        "name": "Lê Văn C",
        "subjects": ["Toán", "Lý"],
        "grades": [9, 10],
        "lat": 21.10,
        "lng": 105.90,
        "hourly_rate": 200_000,
        "profile_completed": 0.7,
        "is_verified_id": False,
        "is_verified_student_card": False,
        "total_classes": 2,
        "completed_classes": 1,
        "avg_rating": 3.9,
        "avg_response_time_sec": 7200,  # 2 tiếng
        "policy_violations_count": 1,
        "free_slots": ["Tue_19-21"],
        "semantic_strength": 0.60
    },
]

# Yêu cầu của phụ huynh
request = {
    "subject": "Toán",
    "grade": 9,
    "lat": 21.02,
    "lng": 105.81,
    "budget_per_session": 150_000,
    "available_slots": ["Mon_19-21", "Wed_19-21"],
    "description": "Cần gia sư Toán lớp 9, ôn thi vào 10, con hơi mất gốc phần hình học"
}

In [3]:
# ==========================
# 2. TRUST SCORE
# ==========================

def calc_trust_score(tutor):
    total = max(tutor["total_classes"], 1)
    completion_rate = tutor["completed_classes"] / total
    avg_rating = tutor["avg_rating"] or 0
    rt = tutor["avg_response_time_sec"] or 9999
    violations = tutor["policy_violations_count"]

    score = 0.0

    # 1. Hồ sơ (0-35)
    profile_score = tutor["profile_completed"] * 20
    if tutor["is_verified_id"]:
        profile_score += 8
    if tutor["is_verified_student_card"]:
        profile_score += 7
    profile_score = min(profile_score, 35)
    score += profile_score

    # 2. Tỷ lệ hoàn thành lớp (0-25)
    if completion_rate >= 0.9:
        completion_score = 25
    elif completion_rate >= 0.75:
        completion_score = 18
    elif completion_rate >= 0.5:
        completion_score = 10
    else:
        completion_score = 5
    score += completion_score

    # 3. Đánh giá phụ huynh (0-25)
    rating_score = (avg_rating / 5.0) * 25
    score += rating_score

    # 4. Thời gian phản hồi (0-10)
    if rt < 300:
        response_score = 10
    elif rt < 3600:
        response_score = 7
    elif rt < 6 * 3600:
        response_score = 4
    else:
        response_score = 2
    score += response_score

    # 5. Tuân thủ chính sách (0-5, có phạt)
    if violations == 0:
        policy_score = 5
    elif violations == 1:
        policy_score = 2
    else:
        policy_score = 0
        score -= 10  # phạt nặng
    score += policy_score

    # Clamp
    score = max(0, min(score, 100))
    return score

In [4]:
# ==========================
# 3. MATCHING ENGINE (RULE + "AI")
# ==========================

def distance_km(lat1, lng1, lat2, lng2):
    # Approx haversine đơn giản (demo)
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlng = math.radians(lng2 - lng1)
    a = (
        math.sin(dlat / 2)**2 +
        math.cos(math.radians(lat1)) *
        math.cos(math.radians(lat2)) *
        math.sin(dlng / 2)**2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def time_overlap_score(request_slots, tutor_slots):
    inter = set(request_slots) & set(tutor_slots)
    if not request_slots:
        return 0.0
    return len(inter) / len(request_slots)

def price_score(budget, rate):
    if rate <= budget:
        return 1.0
    diff = rate - budget
    if diff <= 20_000:
        return 0.8
    elif diff <= 50_000:
        return 0.5
    else:
        return 0.2

def distance_score_km(dist):
    if dist <= 2:
        return 1.0
    elif dist <= 5:
        return 0.8
    elif dist <= 10:
        return 0.5
    else:
        return 0.0


def calc_match_score(request, tutor, trust_score):
    # Các thành phần rule-based
    subj_match = 1.0 if request["subject"] in tutor["subjects"] else 0.0
    grade_match = 1.0 if request["grade"] in tutor["grades"] else 0.0
    overlap = time_overlap_score(request["available_slots"], tutor["free_slots"])
    d = distance_km(request["lat"], request["lng"], tutor["lat"], tutor["lng"])
    dist_score = distance_score_km(d)
    p_score = price_score(request["budget_per_session"], tutor["hourly_rate"])
    trust_norm = trust_score / 100.0

    # Mô phỏng semantic_score từ embeddings:
    # Ở bản thật, bạn tính cosine_similarity(request_embedding, tutor_embedding)
    semantic_score = tutor.get("semantic_strength", 0.5)

    # Kết hợp
    score = (
        0.25 * subj_match +
        0.10 * grade_match +
        0.15 * overlap +
        0.10 * p_score +
        0.10 * dist_score +
        0.15 * trust_norm +
        0.15 * semantic_score
    )
    return score


In [6]:
# ==========================
# 4. CHẠY DEMO
# ==========================

def run_demo():
    print("===== YÊU CẦU CỦA PHỤ HUYNH =====")
    print(request)
    print()

    # Tính Trust Score cho từng gia sư
    print("===== TRUST SCORE CỦA CÁC GIA SƯ =====")
    for t in tutors:
        ts = calc_trust_score(t)
        t["trust_score"] = ts
        print(f"- {t['name']}: Trust Score = {ts:.2f}/100")
    print()

    # Tính Matching Score
    print("===== KẾT QUẢ MATCHING (TOP GIA SƯ PHÙ HỢP) =====")
    results = []
    for t in tutors:
        ms = calc_match_score(request, t, t["trust_score"])
        results.append((ms, t))

    # Sort từ cao xuống thấp
    results.sort(reverse=True, key=lambda x: x[0])

    for rank, (ms, t) in enumerate(results, start=1):
        print(f"#{rank}: {t['name']}")
        print(f"   - Matching Score: {ms:.3f} (0-1)")
        print(f"   - Trust Score:    {t['trust_score']:.2f}/100")
        print(f"   - Giá/buổi:       {t['hourly_rate']:,} VND")
        dist = distance_km(request['lat'], request['lng'], t['lat'], t['lng'])
        print(f"   - Khoảng cách:    {dist:.2f} km")
        print(f"   - Slots trùng:    {set(request['available_slots']) & set(t['free_slots'])}")
        print()

if __name__ == "__main__":
    run_demo()

===== YÊU CẦU CỦA PHỤ HUYNH =====
{'subject': 'Toán', 'grade': 9, 'lat': 21.02, 'lng': 105.81, 'budget_per_session': 150000, 'available_slots': ['Mon_19-21', 'Wed_19-21'], 'description': 'Cần gia sư Toán lớp 9, ôn thi vào 10, con hơi mất gốc phần hình học'}

===== TRUST SCORE CỦA CÁC GIA SƯ =====
- Nguyễn Văn A: Trust Score = 94.00/100
- Trần Thị B: Trust Score = 67.00/100
- Lê Văn C: Trust Score = 49.50/100

===== KẾT QUẢ MATCHING (TOP GIA SƯ PHÙ HỢP) =====
#1: Nguyễn Văn A
   - Matching Score: 0.983 (0-1)
   - Trust Score:    94.00/100
   - Giá/buổi:       150,000 VND
   - Khoảng cách:    1.52 km
   - Slots trùng:    {'Mon_19-21', 'Wed_19-21'}

#2: Trần Thị B
   - Matching Score: 0.846 (0-1)
   - Trust Score:    67.00/100
   - Giá/buổi:       130,000 VND
   - Khoảng cách:    1.11 km
   - Slots trùng:    {'Wed_19-21'}

#3: Lê Văn C
   - Matching Score: 0.564 (0-1)
   - Trust Score:    49.50/100
   - Giá/buổi:       200,000 VND
   - Khoảng cách:    12.90 km
   - Slots trùng:    set()

